# Train a model locally
*Base code from the [Keras Applications](https://keras.io/api/applications/) page.*

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.5.0'

In [39]:
# import libraries from tensorflow keras
from tensorflow.keras import optimizers, losses, activations, models, applications, layers, metrics
from tensorflow.keras.preprocessing import image, image
from tensorflow.keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import numpy as np
import os
import pandas as pd

In [4]:
batch_size = 64
img_height = 558
img_width = 558

In [5]:
data_dir = os.path.dirname(os.getcwd()) + '\\data\\images-model\\inceptionv3\\'

In [75]:
data_dir

'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\'

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode = 'categorical')

Found 2410 files belonging to 2 classes.
Using 1928 files for training.


In [52]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode = 'categorical')

Found 2410 files belonging to 2 classes.
Using 482 files for validation.


In [88]:
# potential culprit
filepaths = val_ds.file_paths

In [89]:
filepaths

['C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG2026-2640-4680.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG8619-10920-3276.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG0354-8652-6180.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\kithara\\IMAG2552-kithara-full.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG10098-10491-7263.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG0053-4128-2064.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG9095-6226-9622.jp

In [87]:
filepaths

['C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG2026-2640-4680.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG8619-10920-3276.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG0354-8652-6180.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\kithara\\IMAG2552-kithara-full.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG10098-10491-7263.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG0053-4128-2064.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG9095-6226-9622.jp

In [9]:
class_names = train_ds.class_names
print(class_names)

['kithara', 'no-kithara']


In [10]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(64, 558, 558, 3)
(64, 2)


In [11]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [13]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.7568628


In [54]:
# potential culprit
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_val_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.23137257 0.4901961


In [15]:
# create the base pre-trained model
# note that if this does not run, downgrade h5py
# pip install "h5py==2.10.0" --force-reinstall
base_model = applications.InceptionV3(weights = 'imagenet',
                                      include_top=False,
                                      input_shape = (558,558,3))

In [16]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

In [17]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [18]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [19]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics = ['BinaryAccuracy',
                        'FalseNegatives',
                        'FalsePositives',
                        'TrueNegatives',
                        'TruePositives',
                        'AUC'])

In [20]:
epochs=2
history = model.fit(
  normalized_ds,
  validation_data = normalized_val_ds,
  epochs=epochs
)

Epoch 1/2
31/31 [==============================] - 695s 22s/step - loss: 0.4248 - binary_accuracy: 0.8091 - false_negatives: 368.0000 - false_positives: 368.0000 - true_negatives: 1560.0000 - true_positives: 1560.0000 - auc: 0.9216 - val_loss: 0.1672 - val_binary_accuracy: 0.9149 - val_false_negatives: 41.0000 - val_false_positives: 41.0000 - val_true_negatives: 441.0000 - val_true_positives: 441.0000 - val_auc: 0.9789
Epoch 2/2
31/31 [==============================] - 672s 22s/step - loss: 0.2439 - binary_accuracy: 0.8791 - false_negatives: 233.0000 - false_positives: 233.0000 - true_negatives: 1695.0000 - true_positives: 1695.0000 - auc: 0.9579 - val_loss: 0.1278 - val_binary_accuracy: 0.9419 - val_false_negatives: 28.0000 - val_false_positives: 28.0000 - val_true_negatives: 454.0000 - val_true_positives: 454.0000 - val_auc: 0.9915


In [21]:
history.history

{'loss': [0.4247855246067047, 0.24385793507099152],
 'binary_accuracy': [0.8091286420822144, 0.8791493773460388],
 'false_negatives': [368.0, 233.0],
 'false_positives': [368.0, 233.0],
 'true_negatives': [1560.0, 1695.0],
 'true_positives': [1560.0, 1695.0],
 'auc': [0.9215623140335083, 0.9579388499259949],
 'val_loss': [0.16720759868621826, 0.12780407071113586],
 'val_binary_accuracy': [0.9149377346038818, 0.9419087171554565],
 'val_false_negatives': [41.0, 28.0],
 'val_false_positives': [41.0, 28.0],
 'val_true_negatives': [441.0, 454.0],
 'val_true_positives': [441.0, 454.0],
 'val_auc': [0.9788528084754944, 0.9914558529853821]}

In [78]:
filepaths

['C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG2026-2640-4680.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG8619-10920-3276.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG0354-8652-6180.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\kithara\\IMAG2552-kithara-full.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG10098-10491-7263.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG0053-4128-2064.jpg',
 'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\no-kithara\\IMAG9095-6226-9622.jp

In [91]:
extracted_values = [int('no-kithara' in f) for f in filepaths]

In [92]:
extracted_values

[1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,


In [90]:
val_ds

<BatchDataset shapes: ((None, 558, 558, 3), (None, 2)), types: (tf.float32, tf.float32)>

In [85]:
# this is the culprit
val_labels = [np.argmax(y.numpy(), axis = -1) for _, y in val_ds]

In [86]:
val_labels

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       dtype=int64),
 array([1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1],
       dtype=int64),
 array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       dtype=int64),
 array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
       dtype=int64),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [56]:
val_labels = np.concatenate(val_labels)

In [57]:
val_labels

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

In [58]:
len(val_labels)

482

In [59]:
# predictions = model.predict(normalized_val_ds)

In [60]:
predictions

array([[1.64201333e-06, 9.99998331e-01],
       [9.95375693e-09, 1.00000000e+00],
       [3.06767644e-04, 9.99693274e-01],
       [8.16505849e-01, 1.83494166e-01],
       [2.23455871e-07, 9.99999762e-01],
       [3.67314428e-08, 1.00000000e+00],
       [2.26904061e-02, 9.77309585e-01],
       [1.87875848e-08, 1.00000000e+00],
       [1.23533368e-07, 9.99999881e-01],
       [3.12712905e-06, 9.99996901e-01],
       [8.77910509e-08, 9.99999881e-01],
       [1.03211607e-06, 9.99998927e-01],
       [3.61470853e-07, 9.99999642e-01],
       [7.69290626e-02, 9.23070908e-01],
       [6.11526193e-04, 9.99388456e-01],
       [9.46905792e-01, 5.30941635e-02],
       [8.64933617e-03, 9.91350651e-01],
       [7.47124673e-09, 1.00000000e+00],
       [5.78445088e-06, 9.99994159e-01],
       [6.75961815e-07, 9.99999285e-01],
       [1.50459260e-01, 8.49540710e-01],
       [9.52820898e-08, 9.99999881e-01],
       [1.82815540e-08, 1.00000000e+00],
       [4.86196939e-07, 9.99999523e-01],
       [2.233525

In [61]:
predictions.min(axis = 0)

array([4.7178657e-09, 4.1068522e-03], dtype=float32)

In [62]:
predicted_class = (predictions[:,1]>0.5).astype(int)

In [63]:
predicted_class

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,

In [64]:
accuracy_score(val_labels, predicted_class)

0.7510373443983402

In [65]:
print(classification_report(val_labels, predicted_class))

              precision    recall  f1-score   support

           0       0.17      0.21      0.19        66
           1       0.87      0.84      0.85       416

    accuracy                           0.75       482
   macro avg       0.52      0.52      0.52       482
weighted avg       0.77      0.75      0.76       482



In [66]:
roc_auc_score(val_labels, predictions[:,1])

0.5230004370629371

In [67]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [93]:
pred_df = pd.DataFrame({'filepath': filepaths, 'actuals': extracted_values, 'predictions': predictions[:,0]})

In [101]:
pd.options.display.col_width(None)

OptionError: 'No such option'

In [94]:
pred_df[pred_df['actuals'] == 0].sort_values('predictions')

,filepath,actuals,predictions
44,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,5.915565e-09
305,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,7.097555e-09
120,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,1.313267e-08
167,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,1.578697e-08
333,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,1.816051e-08
...,...,...,...
108,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,9.522794e-01
145,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,9.526849e-01
159,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,9.807165e-01
245,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,0,9.829969e-01


In [109]:
pred_df.loc[145, 'filepath']

'C:\\Users\\Veronica\\Documents\\2020-21_NSS-DS-Bootcamp\\ds4-cv-vases\\data\\images-model\\inceptionv3\\kithara\\IMAG1554-kithara-full.jpg'

In [108]:
pred_df[pred_df['filepath'].str.contains('IMAG9081')]

,filepath,actuals,predictions
370,C:\Users\Veronica\Documents\2020-21_NSS-DS-Boo...,1,1.480737e-08


In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

### Hand-built model
*Following [this](https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/) tutorial.*

In [ ]:
def define_model():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(558, 558, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = define_model()

In [ ]:
datagen = image.ImageDataGenerator()

In [ ]:
# prepare iterators
train_it = datagen.flow_from_directory('../data/images-model/train/',
    class_mode='binary', batch_size=64, target_size=(558, 558), shuffle = False)
test_it = datagen.flow_from_directory('../data/images-model/test/',
    class_mode='binary', batch_size=64, target_size=(558, 558), shuffle = False)

In [ ]:
# fit model
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=test_it, validation_steps=len(test_it), epochs=2, verbose=0)

In [ ]:
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))

In [ ]:
model.predict(test_it)

In [ ]:
#Confusion Matrix and Classification Report
Y_pred = model.predict_generator(test_it, 603 // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_it.classes, y_pred))
print('Classification Report')
target_names = ['no-kithara', 'kithara']
print(classification_report(test_it.classes, y_pred, target_names=target_names))

In [ ]:
test_data = []
test_labels = []
batch_index = 0

while batch_index <= test_generator.batch_index:
    data = next(test_generator)
    test_data.append(data[0])
    test_labels.append(data[1])
    batch_index = batch_index + 1

test_data_array = np.asarray(test_data)
test_labels_array = np.asarray(test_labels)

### TensorFlow Image Classification Workbook
*From [this](https://www.tensorflow.org/tutorials/images/classification) tutorial.*

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
data_dir = os.path.dirname(os.getcwd()) + '\\data\\images-model\\train\\'

In [ ]:
data_dir

In [ ]:
batch_size = 32
img_height = 558
img_width = 558

In [ ]:
import tensorflow as tf

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
tf.__version__